Домашнее задание по теме «Рекомендации на основе содержания». (№1)

1) Использовать dataset MovieLens

2) Построить рекомендации (регрессия, предсказываем оценку) на фичах:
- TF-IDF на тегах и жанрах
- Средние оценки (+ median, variance, etc.) пользователя и фильма

3) Оценить RMSE на тестовой выборке

In [1]:
# загрузим "Small" dataset: 100,000 ratings
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2022-08-07 08:52:26--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.65MB/s    in 0.2s    

2022-08-07 08:52:26 (4.65 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [2]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [4]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

In [5]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [6]:
links.head(3)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [7]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [8]:
tags.head(3)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992


## Добавим к данным о жанрах информацию о тегах.

In [9]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


## Удалим теги-дубликаты и отсутствующие значения.

In [10]:
movies_with_tags.tag.unique()
movies_with_tags.dropna(inplace=True)

In [11]:
# Общее количество фильмов
movies_with_tags.title.unique().shape

(1572,)

In [12]:
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


## Преобразуем данные о жанрах в строки

In [13]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [14]:
movies_with_tags['genres'] = [change_string(g) for g in movies_with_tags.genres.values]

In [15]:
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure Children Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure Children Fantasy,62.0,magic board game,1.528844e+09


In [16]:
movies_with_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3683 entries, 0 to 9732
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    3683 non-null   int64  
 1   title      3683 non-null   object 
 2   genres     3683 non-null   object 
 3   userId     3683 non-null   float64
 4   tag        3683 non-null   object 
 5   timestamp  3683 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 201.4+ KB


## Посчитаем значения оценок в разрезе пользователей (userId): средние(mean), медианные(median), дисперсию(variance)

In [17]:
# Предварительно удалим ненужные столбцы ('movieId', 'timestamp') из таблицы оценок
ratings_by_user = ratings.drop(['movieId', 'timestamp'], axis=1)

In [18]:
# сгруппируем по 'userId' и посчитаем статистики по каждому пользователю
ratings_by_user = ratings_by_user.groupby(['userId'], as_index=False).agg([np.mean, np.median, np.var])

In [19]:
ratings_by_user.head()

rating                 
            mean median       var
userId                           
1       4.366379    5.0  0.640077
2       3.948276    4.0  0.649015
3       2.435897    0.5  4.370783
4       3.555556    4.0  1.727132
5       3.636364    4.0  0.980973

## Теперь посчитаем то же самое в разрезе фильмов (MovieId): средние(mean), медианные(median), дисперсию(variance)

In [20]:
# Предварительно удалим ненужные колонки ('userId', 'timestamp') из таблицы оценок
ratings_by_movie = ratings.drop(['userId', 'timestamp'], axis=1)

In [21]:
# сгруппируем по 'movieId' и посчитаем статистики по каждому пользователю
ratings_by_movie = ratings_by_movie.groupby(['movieId'], as_index=False).agg([np.mean, np.median, np.var])

In [22]:
ratings_by_movie.reset_index()
ratings_by_movie.columns = ['rating_mean', 'rating_median', 'rating_var']
ratings_by_movie.head()

,rating_mean,rating_median,rating_var
movieId,,,
1,3.920930,4.0,0.696990
2,3.431818,3.5,0.777419
3,3.259615,3.0,1.112651
4,2.357143,3.0,0.726190
5,3.071429,3.0,0.822917


## Объединим все теги по фильмам

In [23]:
def change_string_lower(s):
    return str(s).replace(' ', '').replace('-', '').lower()

tag_strings = []
movies = []

for movie, group in tqdm_notebook(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([change_string_lower(s) for s in group.tag.values]))
    movies.append(movie)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/1572 [00:00<?, ?it/s]

In [24]:
movies[:3]

['(500) Days of Summer (2009)',
 '...And Justice for All (1979)',
 '10 Cloverfield Lane (2016)']

In [25]:
tag_strings[:3]

['artistic funny humorous inspiring intelligent quirky romance zooeydeschanel',
 'lawyers',
 'creepy suspense']

In [26]:
# Положим объединенные теги в датафрейм с названиями фильмов
tags_united = pd.DataFrame({'title': movies, 'tags': tag_strings})
tags_united.head(3)

,title,tags
0,(500) Days of Summer (2009),artistic funny humorous inspiring intelligent ...
1,...And Justice for All (1979),lawyers
2,10 Cloverfield Lane (2016),creepy suspense


## Объединим таблицу фильмов с таблицей сводных тегов по фильму

In [27]:
movies_with_united_tags = movies_with_tags.join(tags_united.set_index('title'), on='title')

In [28]:
movies_with_united_tags.head()

,movieId,title,genres,userId,tag,timestamp,tags
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,336.0,pixar,1.139046e+09,pixar pixar fun
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,474.0,pixar,1.137207e+09,pixar pixar fun
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,567.0,fun,1.525286e+09,pixar pixar fun
1,2,Jumanji (1995),Adventure Children Fantasy,62.0,fantasy,1.528844e+09,fantasy magicboardgame robinwilliams game
1,2,Jumanji (1995),Adventure Children Fantasy,62.0,magic board game,1.528844e+09,fantasy magicboardgame robinwilliams game


## Теперь добавим данные по статистикам в разрезе фильмов (средняя, медиана, дисперсия)

In [29]:
full_data = movies_with_united_tags.join(ratings_by_movie, on='movieId')

In [30]:
full_data.groupby(by='movieId')

In [31]:
full_data.drop_duplicates(subset='movieId', inplace=True)

In [32]:
full_data.head()

,movieId,title,genres,userId,tag,timestamp,tags,rating_mean,rating_median,rating_var
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,336.0,pixar,1.139046e+09,pixar pixar fun,3.920930,4.0,0.696990
1,2,Jumanji (1995),Adventure Children Fantasy,62.0,fantasy,1.528844e+09,fantasy magicboardgame robinwilliams game,3.431818,3.5,0.777419
2,3,Grumpier Old Men (1995),Comedy Romance,289.0,moldy,1.143425e+09,moldy old,3.259615,3.0,1.112651
4,5,Father of the Bride Part II (1995),Comedy,474.0,pregnancy,1.137374e+09,pregnancy remake,3.071429,3.0,0.822917
6,7,Sabrina (1995),Comedy Romance,474.0,remake,1.137376e+09,remake,3.185185,3.0,0.955625


In [33]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1572 entries, 0 to 9732
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movieId        1572 non-null   int64  
 1   title          1572 non-null   object 
 2   genres         1572 non-null   object 
 3   userId         1572 non-null   float64
 4   tag            1572 non-null   object 
 5   timestamp      1572 non-null   float64
 6   tags           1572 non-null   object 
 7   rating_mean    1554 non-null   float64
 8   rating_median  1554 non-null   float64
 9   rating_var     1395 non-null   float64
dtypes: float64(5), int64(1), object(4)
memory usage: 135.1+ KB


In [34]:
# как видим, статистики посчитаны не по всем фильмам, есть пропуски. Заполнять пропуски не будем, просто удалим строки с NaN
full_data.dropna(inplace=True)

In [35]:
# создадим датасет с нужными колонками для обучения модели. Целевая переменная, по всей видимости, это средний рейтинг фильма - 'rating_mean'
data_cleared = full_data[['movieId', 'genres', 'tags', 'rating_median', 'rating_var', 'rating_mean']]

In [36]:
data_cleared.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1395 entries, 0 to 9710
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movieId        1395 non-null   int64  
 1   genres         1395 non-null   object 
 2   tags           1395 non-null   object 
 3   rating_median  1395 non-null   float64
 4   rating_var     1395 non-null   float64
 5   rating_mean    1395 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 76.3+ KB


In [37]:
data_cleared.head()

,movieId,genres,tags,rating_median,rating_var,rating_mean
0,1,Adventure Animation Children Comedy Fantasy,pixar pixar fun,4.0,0.696990,3.920930
1,2,Adventure Children Fantasy,fantasy magicboardgame robinwilliams game,3.5,0.777419,3.431818
2,3,Comedy Romance,moldy old,3.0,1.112651,3.259615
4,5,Comedy,pregnancy remake,3.0,0.822917,3.071429
6,7,Comedy Romance,remake,3.0,0.955625,3.185185


## Теперь всё готово для создания модели. Предварительно векторизуем текстовые признаки: жанры и объединенные теги.

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
# создадим и обучим векторизатор 
vectorizer = TfidfVectorizer()
genres = vectorizer.fit_transform(data_cleared['genres'])

In [40]:
genres

<1395x19 sparse matrix of type '<class 'numpy.float64'>'
	with 3348 stored elements in Compressed Sparse Row format>

In [41]:
tags = vectorizer.fit_transform(data_cleared['tags'])
tags

<1395x1409 sparse matrix of type '<class 'numpy.float64'>'
	with 3355 stored elements in Compressed Sparse Row format>

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [43]:
# целевая переменная
y = data_cleared['rating_mean']

In [44]:
# Попробуем сперва обучить модель с использованием только вектора тегов в качестве входящего набора признаков
# Разделим выборку на трейн и тест 
X_train, X_test, y_train, y_test = train_test_split(tags, y, test_size=0.3, random_state=42)

In [45]:
# создадим и обучим модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [46]:
# рассчитаем прогноз на тестовой выборке и выведем RMSE
pred = model.predict(X_test)
print(mean_squared_error(pred, y_test))

0.30267444054496284


Как видим, использование только тегов не дало хорошего качества предсказаний, т.к. RMSE достаточно большая. Попробуем теперь только вектор жанров в качестве входящего набора признаков.


In [48]:
# Разделим выборку на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(genres, y, test_size=0.3, random_state=42)

In [49]:
# создадим и обучим модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [51]:
# рассчитаем прогноз на тестовой выборке и выведем RMSE
pred = model.predict(X_test)
print(mean_squared_error(pred, y_test))

0.21342561381257558


Ошибка при использовании только вектора жанров уменьшилась (по сравнению с вектором тегов). Возможная причина - в том, что жанры более стандартизированы по сравнению с "произвольными" тегами. Но всё равно RMSE остаётся достаточно большой. Попробуем расширить набор входящих признаков для регрессии, используя из предобработанного датасета (помимо жанров и тегов), ещё и медиану и дисперсию рейтинга по каждому фильму.

In [99]:
X = [tags, genres, data_cleared['rating_median'], data_cleared['rating_var']]

In [100]:
X

[<1395x1409 sparse matrix of type '<class 'numpy.float64'>'
 	with 3355 stored elements in Compressed Sparse Row format>,
 <1395x19 sparse matrix of type '<class 'numpy.float64'>'
 	with 3348 stored elements in Compressed Sparse Row format>,
 0       4.00
 1       3.50
 2       3.00
 4       3.00
 6       3.00
         ... 
 9647    3.75
 9656    3.00
 9692    3.00
 9709    4.00
 9710    4.00
 Name: rating_median, Length: 1395, dtype: float64,
 0       0.696990
 1       0.777419
 2       1.112651
 4       0.822917
 6       0.955625
           ...   
 9647    0.138393
 9656    1.625000
 9692    1.833333
 9709    1.505682
 9710    0.550000
 Name: rating_var, Length: 1395, dtype: float64]

In [101]:
# Разделим выборку на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ValueError: ignored